# Import FMCSA PFDs

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader

# 1. Define the path (adjust if your PDF has a different name)
pdf_path = "../data/raw/fmcsa_op_training.pdf"

# 2. Check if file exists
if os.path.exists(pdf_path):
    print(f"✅ Found file at: {pdf_path}")
    
    # 3. Load the PDF
    try:
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
        print(f"🎉 Success! Loaded {len(pages)} pages.")
        print(f"--- Content of Page 1 ---\n{pages[0].page_content[:500]}...")
    except Exception as e:
        print(f"❌ Error loading PDF: {e}")
else:
    print(f"❌ File NOT found at: {pdf_path}")
    print(f"📂 Current folder is: {os.getcwd()}")
    print(f"📂 Files in parent folder: {os.listdir('..')}")

✅ Found file at: ../data/raw/fmcsa_op_training.pdf
🎉 Success! Loaded 1092 pages.
--- Content of Page 1 ---
eFOTM Compliance Manual      Jan 4th,2 0 2 5
Page  1  
Compliance Manual
For
eFOTM Redevelopment
Federal Motor Carrier Safety Administration (FMCSA)
U.S. Department of Transportation...


# Split the Text into Chunks

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Configure the splitter
# chunk_size=1000: Roughly a few paragraphs
# chunk_overlap=200: Keeps context between cuts so sentences aren't cut in half
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

# 2. Split the document
print("🔪 Splitting the document...")
all_splits = text_splitter.split_documents(pages)

# 3. Check the results
print(f"🎉 Created {len(all_splits)} text chunks from {len(pages)} pages.")
print(f"--- Example Chunk 1 ---\n{all_splits[0].page_content}")
print(f"--- Example Chunk 2 ---\n{all_splits[1].page_content}")

🔪 Splitting the document...
🎉 Created 4404 text chunks from 1092 pages.
--- Example Chunk 1 ---
eFOTM Compliance Manual      Jan 4th,2 0 2 5
Page  1  
Compliance Manual
For
eFOTM Redevelopment
Federal Motor Carrier Safety Administration (FMCSA)
U.S. Department of Transportation
--- Example Chunk 2 ---
eFOTM Compliance Manual                                                                      Jan 4th,2 0 2 5
Page | 2
Table of Contents
1.0 Compliance Manual .......................................................................................................7 
1.1 Stage 1-Monthly Intervention Selection & Carrier Assignment ........................................... 7 
1.1.1 Stage 1 - Introduction ..................................................................................................................... 7 
1.1.2 Safety Measurement System Assesses Carrier Performance, Generates Warning Letters, and 
Tracks Acute and Critical Violations..........................................

# Embed/Save the Chunks

In [3]:
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_chroma import Chroma
import time

# 1. Initialize the Local Model
print("⬇️  Loading FastEmbed model...")
embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")
print("✅ Model loaded!")

# 2. Setup the Database
persist_directory = "../data/chroma_db"
# Initialize the database immediately so we can add to it incrementally
vector_store = Chroma(
    collection_name="fmcsa_regulations",
    embedding_function=embedding_model,
    persist_directory=persist_directory
)

# 3. Define Batch Size (Manageable chunks for your CPU)
batch_size = 100 
total_chunks = len(all_splits)

print(f"🚀 Starting processing of {total_chunks} chunks in batches of {batch_size}...")

# 4. The Loop: Process and Save piece by piece
for i in range(0, total_chunks, batch_size):
    # Get the slice of documents for this batch
    batch = all_splits[i : i + batch_size]
    
    # Add them to the database (this runs the embedding on CPU)
    vector_store.add_documents(documents=batch)
    
    # Progress Update
    current_count = min(i + batch_size, total_chunks)
    print(f"💾 Saved {current_count}/{total_chunks} chunks ({round(current_count/total_chunks*100, 1)}%)")
    
    # Small pause to let your CPU cool down slightly (optional)
    time.sleep(0.5)

print("------------------------------------------------")
print(f"🎉 MISSION COMPLETE! Database fully saved to: {persist_directory}")
print(f"📊 Final Check - Total vectors stored: {vector_store._collection.count()}")

⬇️  Loading FastEmbed model...


c:\Github\fmcsa_assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Model loaded!
🚀 Starting processing of 4404 chunks in batches of 100...
💾 Saved 100/4404 chunks (2.3%)
💾 Saved 200/4404 chunks (4.5%)
💾 Saved 300/4404 chunks (6.8%)
💾 Saved 400/4404 chunks (9.1%)
💾 Saved 500/4404 chunks (11.4%)
💾 Saved 600/4404 chunks (13.6%)
💾 Saved 700/4404 chunks (15.9%)
💾 Saved 800/4404 chunks (18.2%)
💾 Saved 900/4404 chunks (20.4%)
💾 Saved 1000/4404 chunks (22.7%)
💾 Saved 1100/4404 chunks (25.0%)
💾 Saved 1200/4404 chunks (27.2%)
💾 Saved 1300/4404 chunks (29.5%)
💾 Saved 1400/4404 chunks (31.8%)
💾 Saved 1500/4404 chunks (34.1%)
💾 Saved 1600/4404 chunks (36.3%)
💾 Saved 1700/4404 chunks (38.6%)
💾 Saved 1800/4404 chunks (40.9%)
💾 Saved 1900/4404 chunks (43.1%)
💾 Saved 2000/4404 chunks (45.4%)
💾 Saved 2100/4404 chunks (47.7%)
💾 Saved 2200/4404 chunks (50.0%)
💾 Saved 2300/4404 chunks (52.2%)
💾 Saved 2400/4404 chunks (54.5%)
💾 Saved 2500/4404 chunks (56.8%)
💾 Saved 2600/4404 chunks (59.0%)
💾 Saved 2700/4404 chunks (61.3%)
💾 Saved 2800/4404 chunks (63.6%)
💾 Saved 2900/44

In [4]:
# ---------------------------------------------------------
# STEP 4: The First Test (Run this AFTER processing finishes)
# ---------------------------------------------------------

query = "What are the requirements for a driver's daily log?"

print(f"❓ Question: {query}")
print("🔎 Searching database...")

# Search for the 3 most relevant chunks
results = vector_store.similarity_search(query, k=3)

print("\n---------------- RESULTS ----------------")
for i, doc in enumerate(results):
    print(f"\n📄 [Source: {doc.metadata.get('source', 'Unknown')}]")
    print(f"📖 Page: {doc.metadata.get('page', 'Unknown')}")
    print(f"📝 Content Preview: {doc.page_content[:300]}...") # Show first 300 chars
    print("-" * 40)

❓ Question: What are the requirements for a driver's daily log?
🔎 Searching database...

---------------- RESULTS ----------------

📄 [Source: ../data/raw/fmcsa_op_training.pdf]
📖 Page: 606
📝 Content Preview: be cited 395.8(a) or 
395.8(k)(2).  Driver must 
be given the opportunity to 
print current and prior 
seven days RODs at 
roadside. 
Hours of Service for Commercial Motor 
Vehicle Drivers; Regulatory Guidance 
Concerning Records of Duty Status 
Generated by Logging Software Programs 
(79 FR 39342)
...
----------------------------------------

📄 [Source: ../data/raw/fmcsa_op_training.pdf]
📖 Page: 600
📝 Content Preview: device capable of recording a driver’s duty status information accurately and automatically as required by 
49 CFR § 395.15.  The device must be integrally synchronized with specific operations of the CMV in 
which it is installed and, at a minimum, the device must record engine use, road speed, mil...
----------------------------------------

📄 [Source: ../data/raw

In [25]:
import os
import getpass
from dotenv import load_dotenv  # <-- The Safety Tool

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

# 1. SETUP: Reload the Database
print("📂 Re-opening the database connection...")
embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")
persist_directory = "../data/chroma_db"

vector_store = Chroma(
    collection_name="fmcsa_regulations",
    embedding_function=embedding_model,
    persist_directory=persist_directory
)
print("✅ Database loaded!")

# 2. SETUP: Connect to Gemini (SECURELY) 🔒
load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    print("⚠️ No .env file found. Running in Interactive Mode.")
    # The 'getpass' function hides what you type
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
else:
    print("✅ API Key loaded safely from environment.")

# --- THE FIX: Gemini 2.5 Flash ---
print("🧠 Connecting to Gemini 2.5 Flash...")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# 3. BUILD: The Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# 4. ACTION: Ask the Question
query = "What are the requirements for a driver's daily log?"
print(f"\n❓ Question: {query}")
print("🧠 Thinking... (Sending chunks to Gemini 2.5)")

try:
    response = qa_chain.invoke({"query": query})
    
    # 5. Print the Result
    print("\n" + "="*60)
    print("🤖 AI ANSWER:")
    print("="*60)
    print(response["result"])

    print("\n" + "-"*60)
    print("📚 SOURCES USED:")
    for doc in response["source_documents"]:
        source = doc.metadata.get('source', 'Unknown').split('/')[-1]
        print(f"- Page {doc.metadata.get('page')} (File: {source})")
        
except Exception as e:
    print(f"\n❌ Error: {e}")

📂 Re-opening the database connection...
✅ Database loaded!
✅ API Key loaded safely from environment.
🧠 Connecting to Gemini 2.5 Flash...

❓ Question: What are the requirements for a driver's daily log?
🧠 Thinking... (Sending chunks to Gemini 2.5)

🤖 AI ANSWER:
The electronically-generated display and output for a driver's daily log (Records of Duty Status or RODS) must meet the requirements in **49 CFR § 395.8** and are treated as an alternative to paper logs.

Additionally, drivers must be able to provide printouts of their logs at roadside:
*   Drivers must be given the opportunity to print current and prior seven days of RODS at roadside if cited under 395.8(a) or 395.8(k)(2).
*   If using logging software and an application device without electronic signature capabilities, the driver must be given the opportunity to print the current day's RODS at roadside.

------------------------------------------------------------
📚 SOURCES USED:
- Page 606 (File: fmcsa_op_training.pdf)
- Page 

In [23]:
def ask_fmcsa(question):
    """
    Takes a question, searches the FMCSA database, and returns a sourced answer.
    (Includes 'Off-By-One' fix for page numbers)
    """
    print(f"\n❓ Question: {question}")
    print("🧠 Thinking...")
    
    try:
        # Run the chain
        response = qa_chain.invoke({"query": question})
        
        # Print the Answer
        print("\n" + "🤖 AI ANSWER:")
        print("-" * 20)
        print(response["result"])
        
        # Print Sources
        print("\n" + "📚 SOURCES:")
        seen_sources = set()
        for doc in response["source_documents"]:
            source = doc.metadata.get('source', 'Unknown').split('/')[-1]
            
            # --- THE FIX IS HERE ---
            # Grab the raw page number (default to -1 if missing)
            raw_page = doc.metadata.get('page', -1)
            
            # Add 1 to convert "Index 0" to "Page 1"
            human_page = raw_page + 1
            
            # De-duplicate references
            ref = f"{source} (Page {human_page})"
            if ref not in seen_sources:
                print(f"- {ref}")
                seen_sources.add(ref)
                
    except Exception as e:
        print(f"❌ Error: {e}")


In [13]:
# --- TEST IT OUT ---
# Now you can ask anything instantly!
ask_fmcsa("What defines a 'preventable accident'?")


❓ Question: What defines a 'preventable accident'?
🧠 Thinking...

🤖 AI ANSWER:
--------------------
Based on the provided text, an accident is considered **preventable** if there is any uncertainty in a report as to whether the driver could have avoided the accident.

Key points defining a preventable accident:
*   The determination focuses on whether the driver *could have avoided* the accident, not necessarily who "caused" it. These are separate issues.
*   The motor carrier must present "compelling evidence" to prove an accident was *not* preventable.
*   If police and insurance company reports fail to indicate when the driver became aware of a dangerous situation or how much time they had to react effectively, and there's uncertainty about avoidance, the accident must be considered preventable.

📚 SOURCES:
- fmcsa_op_training.pdf (Page 801)
- fmcsa_op_training.pdf (Page 115)


In [14]:
ask_fmcsa("What is considered evidence of a preventable accident?")


❓ Question: What is considered evidence of a preventable accident?
🧠 Thinking...

🤖 AI ANSWER:
--------------------
The provided text does not explicitly list what is considered "evidence of a preventable accident." Instead, it outlines the criteria and evidence required to determine if an accident is **non-preventable**.

However, based on the context, an accident is considered preventable if:

*   There is **any uncertainty** in a report as to whether the driver could have avoided the accident.
*   The motor carrier fails to present "compelling evidence" on the issue of preventability (i.e., compelling evidence that the accident was non-preventable).
*   Police and insurance company reports, which often fall short of the "compelling evidence" standard, do not clearly indicate when the driver became aware of a dangerous situation or had enough time to react effectively to avoid the accident.

In essence, if the evidence does not compellingly prove that the driver could **not** have a

In [15]:
ask_fmcsa("What are the main safety factors or pillars according to FMCSA?")


❓ Question: What are the main safety factors or pillars according to FMCSA?
🧠 Thinking...

🤖 AI ANSWER:
--------------------
Based on the provided text, the FMCSA's main safety goals and methods, particularly through the COMPASS program, focus on:

*   **Saving lives and improving the safety of commercial motor vehicles.**
*   **Creating a single source for crucial safety data** via single sign-on access.
*   **Improving data quality** to enable better, more informed decision making.
*   **Providing actionable information and data.**
*   **Optimizing FMCSA's business processes and improving the Agency's IT functionality.**

📚 SOURCES:
- fmcsa_op_training.pdf (Page 314)
- fmcsa_op_training.pdf (Page 979)


In [24]:
ask_fmcsa("Provide me with some examples of documents that may be used to prove violations of part 391")


❓ Question: Provide me with some examples of documents that may be used to prove violations of part 391
🧠 Thinking...

🤖 AI ANSWER:
--------------------
Here are some examples of documents that may be used to prove violations of Part 391:

*   Statement from a motor carrier official, driver, or other person responsible for compliance with Part 391.
*   DQ Worksheet (potentially CAPRI DQ Worksheet), verified by a motor carrier official or other person responsible for compliance with Part 391.
*   Driver’s RODS (Records of Duty Status) and corresponding shipping paper/bill of lading.
*   Vehicle registration showing GVWR or other documentary evidence proving that the vehicle was subject to Part 391.
*   A statement from the motor carrier attesting to missing documents or, if applicable, utilizing a DQ Worksheet (or CAPRI DQ Worksheet) to have the motor carrier verify the lack of documents, if copies of documents/certificates required by Part 391 were unavailable or do not exist.
*   Cer